# TEMPLATE

## Engagement Notes

# Enumeration

### Set variables for further engagement

In [ ]:
import ipyparallel as ipp
from pprint import pprint

source =! ip address | grep tun | grep 10 | tr "/" " " | awk '{print $2}'
target = '192.168.20.202'

print(f"source: {source}")
print(f"target: {target}")

### Port scan target

In [ ]:
#!docker run -it --rm -v $(pwd):/app/target rustscan -a $target
from multiprocess import Process
from os import system

def scan(target):
    system(f"docker run --rm -v $(pwd):/app/target rustscan -a {target}")

scan = Process(target=scan, args=(target,))
scan.start()

### URL scan target

In [ ]:
#!gobuster dir -u http://$target -w /usr/share/wordlists/dirbuster/directory-list-lowercase-2.3-small.txt -x txt,js,html,php -t 40 --timeout=6s -o gobuster-task.txt --retry
protocol = 'http'
port = ':8080'

def gobuster(protocol, target, port):
    system(f"gobuster dir -u {protocol}://{target}{port} -w /usr/share/wordlists/dirbuster/directory-list-lowercase-2.3-small.txt -x txt,js,html,php -t 40 --timeout=6s -o gobuster-task.txt --retry")

gobuster = Process(target=gobuster, args=(protocol, target, port))
gobuster.start()

# Exploit

## Local HTTP server

In [ ]:
# Need all this to run a forked server but keep the output 
# where I can see it while still being able to execute other cells.
import psutil
import socket
from time import sleep
from multiprocess import Process
from http.server import HTTPServer, SimpleHTTPRequestHandler

def server():
    pids = [conn.pid for conn in psutil.net_connections() if conn.laddr.port == 8080]
    if len(pids) > 0:  
        psutil.Process(pids[0]).terminate()
        sleep(1) # Need a little time to kill process before trying to use port again
    httpd = HTTPServer(('localhost', 8080), SimpleHTTPRequestHandler)
    httpd.serve_forever()

server = Process(target=server, args=())
server.start()

## pwntools

In [ ]:
from pwn import *

# Set up pwntools for the correct architecture
context.update(arch='amd64')
context.terminal = ['/usr/bin/konsole', '--new-tab', '-e']
exe = './'
elf = context.binary = ELF(exe, checksec=False)

def start(argv=[], *a, **kw):
    return process([exe] + argv, *a, **kw)

#===========================================================
#                    EXPLOIT GOES HERE
#===========================================================

io = start()

io.close()


## SSH Establish

In [ ]:
# set up paramiko ssh client
user = {
    'username': '',
    'password': ''
}
import paramiko
ssh_user = paramiko.SSHClient()

# set up ssh client
ssh_user.set_missing_host_key_policy(paramiko.AutoAddPolicy())

# connect to ssh
ssh_user.connect(target, username=user['username'], password=user['password'])

def user(command):
    stdin, stdout, stderr = ssh_lennie.exec_command(command)
    return stdout.readlines()

# Post Notes